In [0]:
from pyspark.sql.functions import *

In [0]:
RAW_SCHEDULES = "/Volumes/rail_lakehouse/bronze_rail/raw_files/schedules"
CHECKPOINT_SCHEDULES = "/Volumes/rail_lakehouse/bronze_rail/checkpoints/schedules"

print(f"📂 Raw schedules: {RAW_SCHEDULES}")
print(f"💾 Checkpoint: {CHECKPOINT_SCHEDULES}")

In [0]:
df_bronze_schedules = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", f"{CHECKPOINT_SCHEDULES}/schema")
    .option("cloudFiles.schemaEvolutionMode", "rescue")
    .load(RAW_SCHEDULES)
)

In [0]:
df_bronze_clean = (
    df_bronze_schedules
    .select(
        col("trainNumber").alias("train_number"),
        col("trainType").alias("train_type"),
        col("timeTableRows")
    )
    .withColumn("ingest_ts", current_timestamp())
)

query_bronze_schedules = (
    df_bronze_clean.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", f"{CHECKPOINT_SCHEDULES}/bronze")
    .trigger(availableNow=True)
    .table("rail_lakehouse.bronze_rail.bronze_schedules")
)

query_bronze_schedules.awaitTermination()

In [0]:
%sql
SELECT * 
FROM rail_lakehouse.bronze_rail.bronze_schedules 
LIMIT 5;